In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import print_function
import collections
import math
import os
import random
import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import sys
import time
import json
import requests
import lxml.html
import io
from lxml.cssselect import CSSSelector
import pandas as pd
import yaml


YOUTUBE_COMMENTS_URL = 'https://www.youtube.com/all_comments?v={youtube_id}'
YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/comment_ajax'

USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'

comments=0
def find_value(html, key, num_chars=2):
    pos_begin = html.find(key) + len(key) + num_chars
    pos_end = html.find('"', pos_begin)
    return html[pos_begin: pos_end]


def extract_comments(html):
    tree = lxml.html.fromstring(html)
    item_sel = CSSSelector('.comment-item')
    text_sel = CSSSelector('.comment-text-content')
    time_sel = CSSSelector('.time')
    author_sel = CSSSelector('.user-name')

    for item in item_sel(tree):
        yield {'cid': item.get('data-cid'),
               'text': text_sel(item)[0].text_content(),
               'time': time_sel(item)[0].text_content().strip(),
               'author': author_sel(item)[0].text_content()}


def extract_reply_cids(html):
    tree = lxml.html.fromstring(html)
    sel = CSSSelector('.comment-replies-header > .load-comments')
    return [i.get('data-cid') for i in sel(tree)]


def ajax_request(session, url, params, data, retries=10, sleep=20):
    for _ in range(retries):
        response = session.post(url, params=params, data=data)
        if response.status_code == 200:
            response_dict = json.loads(response.text)
            return response_dict.get('page_token', None), response_dict['html_content']
        else:
            time.sleep(sleep)


def download_comments(youtube_id, sleep=1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT

    # Get Youtube page with initial comments
    response = session.get(YOUTUBE_COMMENTS_URL.format(youtube_id=youtube_id))
    html = response.text
    reply_cids = extract_reply_cids(html)

    ret_cids = []
    for comment in extract_comments(html):
        ret_cids.append(comment['cid'])
        yield comment

    page_token = find_value(html, 'data-token')
    session_token = find_value(html, 'XSRF_TOKEN', 4)

    first_iteration = True

    # Get remaining comments (the same as pressing the 'Show more' button)
    while page_token:
        data = {'video_id': youtube_id,
                'session_token': session_token}

        params = {'action_load_comments': 1,
                  'order_by_time': True,
                  'filter': youtube_id}

        if first_iteration:
            params['order_menu'] = True
        else:
            data['page_token'] = page_token

        response = ajax_request(session, YOUTUBE_COMMENTS_AJAX_URL, params, data)
        if not response:
            break

        page_token, html = response

        reply_cids += extract_reply_cids(html)
        for comment in extract_comments(html):
            if comment['cid'] not in ret_cids:
                ret_cids.append(comment['cid'])
                yield comment

        first_iteration = False
        time.sleep(sleep)

    # Get replies (the same as pressing the 'View all X replies' link)
    for cid in reply_cids:
        data = {'comment_id': cid,
                'video_id': youtube_id,
                'can_reply': 1,
                'session_token': session_token}

        params = {'action_load_replies': 1,
                  'order_by_time': True,
                  'filter': youtube_id,
                  'tab': 'inbox'}

        response = ajax_request(session, YOUTUBE_COMMENTS_AJAX_URL, params, data)
        if not response:
            break

        _, html = response

        for comment in extract_comments(html):
            if comment['cid'] not in ret_cids:
                ret_cids.append(comment['cid'])
                yield comment
        time.sleep(sleep)

def main():
  youtube_id = "zH5btDbjLkc"
  limit = 6300

  a=[]
  for i in range(0,limit):
    a.append(0)
  print('Downloading Youtube comments for video:', youtube_id)
  count=0

  for comment in download_comments(youtube_id):
    a[count]=(json.dumps(comment, ensure_ascii=True))
    a[count]=yaml.load(a[count])
    a[count]=a[count]
    b=a[count]
    for k,v in b.items():
      b[k] = "".join(map(lambda c: c if 32<ord(c)<127 else " " , v)) #removing non ascii char from a dictionary
    a[count]=b
    count += 1
    sys.stdout.write('Downloaded %d comment(s)\r' % count)
    sys.stdout.flush()
    if limit and count >= limit:
      break
  print('\nDone!')


  j=[]
  for i in range (0,limit):
    j.append(pd.DataFrame([a[i]]))
  df=pd.concat(j)


  columns_for_differencing = ['cid','time','author','Index']
  df= df.copy()[df.columns.difference(columns_for_differencing)]
  df = df.astype(str)
  b=np.array(df[1:limit])

  d=[]
  for i in range(0,len(b)):
    d.append(0)

  for j in range(0,len(b)):
    for i in b[j]:
        d[j]=str(i)
  with open('comments.txt', 'a') as f:
    for item in d:
        f.write("%s\n" % item)
  words1=[]
  words =[]
  stopwords=[]
  os.chdir("/home/her/tourist review/latest")
  filename = 'stopwords.txt'
  file = open(filename,"r" )
  lines = file.read().splitlines()
  stopwords=lines
  print(stopwords)
      

  for i in range(0,len(d)):
    words1=d[i].split()
    for j in range(0,len(words1)):
        words.append(words1[j])
  wordsf=[]
  words = [word.lower() for word in words]
  words = [word for word in words if word.isalpha()]
  wordsf=(set(words).difference(stopwords))           
  print(wordsf)
  
#   return(words) 

if __name__ == "__main__":
	main()

  



LABELLING SECTION

In [0]:
import os
os.chdir("/home/her/tourist review/latest")

file = open('comments.txt',"r") #comments of  delhi video tours  especially food related
d= file.read().splitlines()  
print(len(d),"of comments in the given dataset")
file.close()

 

filename = 'negative.txt'
file = open(filename,"r" )
negative = file.read().splitlines()
file.close()   

filename = 'positive.txt'
file = open(filename,"r" )
positive = file.read().splitlines()
file.close()

lab=[]
for i in range(len(d)):
    lab.append(0)
def label(d,negative,positive):
    for i in range(len(d)):
        words=d[i].split()
        for j in range(len(words)):
            for k in range(len(positive)):
                if(words[j]==positive[k]):
                    lab[i]=1
                
            for l in range(len(negative)): 
                if(words[j]==negative[l]):
                    lab[i]=-1
                
           
label(d,negative,positive)    
print(lab)


In [0]:
with open('label.txt', 'w') as f:
    for item in lab:
        f.write("%s\n" % item)


In [0]:
len(lab)
len(d)

56691

In [0]:

filename = 'stopwords.txt'
file = open(filename,"r" )
stopwords = file.read().splitlines()
file.close() 

words1=[]
words =[]
for i in range(0,len(d)):
    words1=d[i].split()
    for j in range(0,len(words1)):   
        if(len(words1[j])<9):         #part of filtering
            words.append(words1[j])
        
wordsf=[]
words = [word.lower() for word in words]
words = [word for word in words if word.isalpha()]
wordsf=(set(words).difference(stopwords)) 
print(len(wordsf))
words=list(wordsf)


16853


In [0]:
count=[]
for i in range(len(words)):
    count.append(0)
for i in range(len(words)):
    for j in range(len(words)):
        if(words[i]==words[j]):
            count[i]+=1
            
print(count)

In [0]:
print(words)